In [ ]:
import numpy as np
import torch
from torch.utils import data

In [ ]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    return data.DataLoader(data.TensorDataset(*data_arrays), batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [ ]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))
# 初始化模型
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

In [ ]:
# 损失函数
# loss = nn.MSELoss()
loss = nn.HuberLoss()

In [ ]:
# 小批量随机梯度下降算法(SGD)
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')